In [3]:
import evebox as box
import pandas as pd
import tensorflow as tf
import cProfile
from tqdm.notebook import tqdm, trange

In [4]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.5).with_market_types()

In [48]:
#orders = box.load_orders(uni, tqdm = tqdm)
orders = pd.read_csv('market/current.csv')

In [4]:
#orders.to_csv('market/current.csv')

In [49]:
# Filter all buy orders which are 50x above the median
median   = orders.groupby('type_id')[['price']].median().rename(columns = {'price' : 'median_price'})

orders = orders[
    (50 * orders.merge(median, on = 'type_id')['median_price'] >= orders['price']) |
    ~orders['is_buy_order']
]

In [50]:
# Filter all orders that can not be handled profitably
max_buy  = orders[ orders['is_buy_order']].groupby('type_id')[['price']].max().rename(columns = {'price' : 'max_buy'})
min_sell = orders[~orders['is_buy_order']].groupby('type_id')[['price']].min().rename(columns = {'price' : 'min_sell'})

orders = orders.merge(max_buy, on = 'type_id').merge(min_sell, on = 'type_id')

orders = orders[
    (
        # Sell orders must be below max_buy
        orders['is_buy_order'] |
        (orders['price'] <= orders['max_buy'])
    ) & 
    (
        # Buy orders must be above min_sell
        ~orders['is_buy_order'] |
        (orders['price'] >= orders['min_sell'])
    )
]

In [56]:
# Reduce universe types to all types appearing in the order list

# Pre-filter types
def check_type(t):
    if t["volume"] > 155000:
        return False
    
    return t['type_id'] in orders['type_id']

# Reduce universe
types = [t for t, v in tqdm(uni.types.items()) if check_type(v)]
print('Reducing from {} to {} types'.format(len(uni.types), len(types)))
uni = uni.with_types(types)

# Reduce orders
orders = orders[orders['type_id'].isin(uni.types)]

1268



1268


In [59]:
class Model(tf.keras.Model):
    def __init__(self, universe):
        super().__init__()
        
        self.bandwidth = 128
        self.d_notes  = 4
        self.memory = 64
        
        self.embedding = box.tf.Embedding(universe, d_notes = self.d_notes, tqdm = tqdm)
        
        self.input_transforms = {
            k : tf.keras.layers.Dense(self.bandwidth)
            for k in ['state', 'orders', 'cargo', 'systems']
        }
        
        def otf(n):
            return tf.keras.layers.Dense(n)
        
        self.output_transforms = {
            'actions' : otf(3),
            'move_targets' : otf(1),
            'buy_params' : otf(3),
            'sell_params' : otf(3),
            'value' : otf(1)
        }
        
        self.stack_start = box.tf.StackedTogether(
            tf.keras.Sequential([
                box.tf.MultiHeadedAttention(self.bandwidth, 16, 8) for i in range(2)
            ])
        )
        
        self.stack_step = box.tf.StackedTogether(
            tf.keras.Sequential([
                box.tf.MultiHeadedAttention(self.bandwidth, 16, 8) for i in range(2)
            ])
        )
        
        self.rnn_cell = tf.keras.layers.GRUCell(self.bandwidth)
            
    
    def _preprocess_input(self, input):
        # Pre-process input (adds "note" variables to types & systems, joins orders with their types & systems)
        input = self.embedding(input)
        
        # Expand state to have the same shape as the other stuff
        input['state'] = tf.expand_dims(input['state'], axis = -2)
        
        # Expand all items into a [...,bandwidth] shape
        input = {
            k : self.input_transforms[k](v)
            for k, v in input.items()
        }
        
        return input
        
    def get_initial_state(self, input):
        input = self._preprocess_input(input)
        
        input = self.stack_start(input)
        
        return [tf.reshape(input['state'], [-1, self.bandwidth])]
    
    def call(self, input):
        input, rnn_state = input
        
        batch_shape = tf.shape(input['state'])[:-1]
        
        input = self._preprocess_input(input)
        
        # Add RNN state to the mix
        input['rnn_state'] = tf.reshape(rnn_state, tf.concat([batch_shape, [1, self.bandwidth]], axis = 0))
        
        # We don't want orders in here, too costly (but probably not present anyway)
        if 'orders' in input:
            del input['orders']
        
        # Apply attention stack
        input = self.stack_step(input)
        
        # Extract rnn state & apply to cell
        rnn_in = tf.reshape(input['rnn_state'], [-1, self.bandwidth])
        _, rnn_state = self.rnn_cell(rnn_in, rnn_state)
        del input['rnn_state']
        
        output = {
            'actions' : tf.squeeze(
                self.output_transforms['actions'](input['state']),
                axis = -2
            ),
            
            'move_targets' : tf.squeeze(
                self.output_transforms['move_targets'](input['systems']),
                axis = -1
            ),
            
            'buy_params'  : self.output_transforms['buy_params'] (input['cargo']),
            'sell_params' : self.output_transforms['sell_params'](input['cargo']),
            
            'value' : tf.squeeze(
                self.output_transforms['value'](input['state']),
                axis = [-2, -1]
            )
        }
        
        return output, rnn_state
        
        
        

model = Model(uni)

In [57]:
gym = box.tf.TradingGym(uni, orders)

In [58]:
jita = [s["system_id"] for s in uni.systems.values() if s["name"] == "Jita"][0]

state = box.MutableState()
state.time_left = 100.0
state.system    = jita

state = box.State(state)

In [ ]:
unroller = gym.unroll_model(model, tqdm);

opt = tf.keras.optimizers.SGD(1e-5)

unroller(state, 1)

def get_loss():
    result = unroller(state, 100)
    
    policy_loss, value_loss, entropy_loss = gym.losses(result)
    
    total_loss = policy_loss + value_loss + 1e-3 * entropy_loss
    tf.print(policy_loss)
    tf.print(value_loss)
    tf.print(entropy_loss)
    tf.print(total_loss)
    return total_loss

def mean_loss():
    losses = tf.stack(
        [get_loss() for i in tqdm(range(1), desc = 'Batch', leave = False)],
        axis = -1
    )
    
    mean_loss = tf.math.reduce_mean(losses)
    
    return mean_loss

for i in trange(100, desc = 'Iterations'):
    print(opt.minimize(mean_loss, model.trainable_variables))

In [13]:
states, actions, logps, values = zip(*result)

actions

(Buy 61411932.0 of 42241,
 Buy 10420804.0 of 11567,
 Buy 41306360.0 of 11567,
 Buy 56765748.0 of 11567,
 Buy 25071562.0 of 42241,
 Buy 71965600.0 of 42241,
 Buy 69669368.0 of 42241,
 Buy 70687472.0 of 11567,
 Buy 44765412.0 of 42241,
 Buy 31007356.0 of 42241)